### Heroes Of Pymoli Data Analysis
1. Of the 576 active players, the vast majority are `male (84%)`. There also exists, a smaller, but notable proportion of `female players (14%)`.


2. While many palyers are male, the average spending per person does not seem to vary depending on gender. Avearge spending per person: `Male ($ 4.07)`, `Female` ($4.47), `Other ($4.56)`.


3. The vast majortiry of the players age between `15 - 30 years old`. The peak age demographic falls between `20-24 (44.8%)` with secondary groups falling between `15-19 (18.60%)` and `25-29 (13.4%)`.  


3. Purchase behavior by age. It's obvious that the age group `20-24 (44.8%)` has the most players, so they contribute the most to the total revenue. But interstingly, there is a increasing purchasing trend for age group above 30 years old, both the `average spending per person` and the `average purchased item price` increases for the age group `30-34` and `35-39`. Perhaps due to the increasing purchase capability. 
-----

### Detail Analysis

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

print("File loaded!")
print("There are {} rows to analyze.".format(purchase_data.count().iloc[0]))

File loaded!
There are 780 rows to analyze.


In [2]:
print(purchase_data.columns)
purchase_data.head()

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'], dtype='object')


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [3]:
num_player = len(purchase_data['SN'].unique())
num_player_df = pd.DataFrame({"Total Players": [num_player]})
num_player_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
# Obtain basic stats for the game
item_price_df = purchase_data[["Item ID", "Price"]].copy()
item_price_df = item_price_df.drop_duplicates()

unique_items = item_price_df["Item ID"].count()
average_price = item_price_df["Price"].sum() / unique_items
num_purchase = len(purchase_data["Purchase ID"].unique())
revenue = purchase_data["Price"].sum()

# create data frame for summary
item_sale_summary_df = pd.DataFrame({"Number of Unique Items" : [unique_items],
                                    "Average Price": ["${:.2f}".format(average_price)],
                                    "Number of Purchases" : [num_purchase],
                                    "Total Revenue" : ["${:,.2f}".format(revenue)] })
item_sale_summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.04,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [6]:
gender_df = purchase_data[["SN", "Gender"]].drop_duplicates() # drop duplicate for multiple purchase from same players

# group DataFrame by "Gender", and count number of player  
gender_df = gender_df.groupby("Gender").count()
gender_df = gender_df.rename(columns = {"SN" : "Total Count"})
gender_df["Percentage of Players"] = gender_df["Total Count"] / num_player * 100
gender_df["Percentage of Players"] = gender_df["Percentage of Players"].map("{:.2f} %".format)
gender_df

,Total Count,Percentage of Players
Gender,,
Female,81,14.06 %
Male,484,84.03 %
Other / Non-Disclosed,11,1.91 %



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
# create groupped object
purchase_gender_grouped = purchase_data.groupby("Gender")

#create empty DataFrmae for gender analysis summary 
purchase_gender_df = pd.DataFrame()
purchase_gender_df["Purchase Count"] = purchase_gender_grouped.count()["Purchase ID"]
purchase_gender_df["Average Purchase Price"] = purchase_gender_grouped.mean()["Price"]
purchase_gender_df["Total Purchase Price"] = purchase_gender_grouped.sum()["Price"]
purchase_gender_df["Avg Total Purchase per Person"] = purchase_gender_df["Total Purchase Price"] / \
                                                      gender_df["Total Count"]

# Formatting
cols = ["Average Purchase Price", "Total Purchase Price", "Avg Total Purchase per Person"]
for col in cols:
    purchase_gender_df[col] = purchase_gender_df[col].map("${:,.2f}".format)

purchase_gender_df

,Purchase Count,Average Purchase Price,Total Purchase Price,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [8]:
# create bins and label names for age analysis 
# [0, 9, 14, 19, 24, 29, 34, 39, inf] -- instead of hard coding, each bin is off by 5
bins = [x for x in range(9, 40, 5)]
label_name = [str(bins[i] + 1) + "-" + str(bins[i + 1]) for i in range(len(bins) - 1)]

bins = [0] + bins + [np.inf]
label_name = ["<10"] + label_name + ["40+"]
print(bins)
print(label_name)

[0, 9, 14, 19, 24, 29, 34, 39, inf]
['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']


In [9]:
# sort the purchase data frame by age, so that after groupping, the age_group label can be shown in a ascending order
purchase_data_AgeSorted = purchase_data.sort_values("Age").reset_index(drop = True)

# create age group by binning 
purchase_data_AgeSorted["Age group"] = pd.cut(purchase_data_AgeSorted["Age"], bins, labels = label_name)

# creating Age analysis DataFrame by selecting proper columns
age_df = purchase_data_AgeSorted[["SN", "Age", "Age group"]].copy().drop_duplicates()
age_df.head()

,SN,Age,Age group
0,Haillyrgue51,7,<10
1,Sisur91,7,<10
2,Chanossast57,7,<10
4,Quilassa66,7,<10
5,Eusri44,7,<10


In [10]:
# counting player at each age group
age_count = age_df.groupby("Age group").count()["Age"]
age_count = pd.DataFrame(age_count).rename(columns = {"Age" : "Total Count"})

# calculate percentage + formatting 
age_count["Percentage of Players"] = (age_count["Total Count"] / num_player * 100).map("{:.2f}%".format)

del age_count.index.name
age_count

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [11]:
purchase_data_AgeSorted.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age group
0,33,Haillyrgue51,7,Male,44,Bonecarvin Battle Axe,2.38,<10
1,778,Sisur91,7,Male,101,Final Critic,4.19,<10
2,446,Chanossast57,7,Female,119,"Stormbringer, Dark Blade of Ending Misery",4.32,<10
3,515,Haillyrgue51,7,Male,40,Second Chance,2.52,<10
4,658,Quilassa66,7,Female,178,"Oathbreaker, Last Hope of the Breaking Storm",4.23,<10


In [12]:
purchase_age_group = purchase_data_AgeSorted.groupby("Age group")

# create empty DataFrame for age analysis
purchase_age_df = pd.DataFrame()
# fill in contents
purchase_age_df["Purchase Count"] = purchase_age_group.count()["Price"]
purchase_age_df["Average Purchase Made per Person"] = (purchase_age_df["Purchase Count"] / 
                                                       age_count["Total Count"]).map("{:.2f}".format)
purchase_age_df["Average Purchase Price"] = purchase_age_group.mean()["Price"]
purchase_age_df["Total Purchase Values"] = purchase_age_group.sum()["Price"]
purchase_age_df["Average Total Purchase per Person"] = (purchase_age_df["Total Purchase Values"] / 
                                                        age_count["Total Count"])

# Formatting
cols = ["Average Purchase Price", "Total Purchase Values", "Average Total Purchase per Person"]
for col in cols:
    purchase_age_df[col] = purchase_age_df[col].map("${:.2f}".format)
    
purchase_age_df

,Purchase Count,Average Purchase Made per Person,Average Purchase Price,Total Purchase Values,Average Total Purchase per Person
Age group,,,,,
<10,23,1.35,$3.35,$77.13,$4.54
10-14,28,1.27,$2.96,$82.78,$3.76
15-19,136,1.27,$3.04,$412.89,$3.86
20-24,365,1.41,$3.05,$1114.06,$4.32
25-29,101,1.31,$2.90,$293.00,$3.81
30-34,73,1.40,$2.93,$214.00,$4.12
35-39,41,1.32,$3.60,$147.67,$4.76
40+,13,1.08,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [13]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [14]:
purchase_player_grouped = purchase_data.groupby("SN")

# creating empty DataFrame for individual players analysis
purchase_player_df = pd.DataFrame()
purchase_player_df["Purchase Count"] = purchase_player_grouped.count()["Item Name"]
purchase_player_df["Average Purchase Price"] = (purchase_player_grouped.sum()["Price"] / 
                                                purchase_player_df["Purchase Count"])
purchase_player_df["Total Purchase Price"] = purchase_player_grouped.sum()["Price"]

# sort the DataFrame by "Total Purchase Price"
purchase_player_df = purchase_player_df.sort_values(by = ["Total Purchase Price"], ascending = False)

# Formatting 
cols = ["Average Purchase Price", "Total Purchase Price"]
for col in cols:
    purchase_player_df[col] = purchase_player_df[col].map("${:.2f}".format)

purchase_player_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Price
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [15]:
purchase_item_group = purchase_data.groupby(["Item ID", "Item Name"])

# create DataFrame for item analysis
purchase_item_df = pd.DataFrame()
purchase_item_df["Purchase Count"] = purchase_item_group.count()["Price"]
purchase_item_df["Item Price"] = purchase_item_group.mean()["Price"]
purchase_item_df["Total Purchase Value"] = purchase_item_df["Item Price"] * purchase_item_df["Purchase Count"]

# sort DataFrame based on popularity -- "Purchase Count"
purchase_item_df = purchase_item_df.sort_values(by = ["Purchase Count"], ascending = False)

cols = ["Item Price", "Total Purchase Value"]
item_df = purchase_item_df.copy()
# Formatting
for col in cols:
    item_df[col] = purchase_item_df[col].map("${:.2f}".format)
    
item_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [16]:
purchase_item_df = purchase_item_df.sort_values(by = "Total Purchase Value", ascending = False)

cols = ["Item Price", "Total Purchase Value"]
most_profitable_item = purchase_item_df.copy()

# Formatting
for col in cols:
    most_profitable_item[col] = purchase_item_df[col].map("${:.2f}".format)
    
most_profitable_item.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
